In [1]:
import torch
import torchvision as vision
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
datasettrain = vision.datasets.CIFAR100("data",True,vision.transforms.ToTensor(),download=True)

datasettest = vision.datasets.CIFAR100("data",False,vision.transforms.ToTensor(),download=True)
datagen = torch.utils.data.DataLoader(datasettrain,10,True)
datatestgen = torch.utils.data.DataLoader(datasettest,10,True)
for X,y in datagen:
    print(X[0].shape)
    print(y.shape)
    break
# temp = torch.utils.data.DataLoader(datasettrain,999999,True)
# for x,y in temp:
#     print(np.unique(y).shape)
#     print(y.shape)
#     break

#(100,)
#torch.Size([50000])

# nn.functional.one_hot(y,100).shape
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv2d = nn.Sequential(
            nn.Conv2d(3,32,2,padding=1),
            nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            
            torch.nn.MaxPool2d(2),
            
            nn.Conv2d(32,64,2,padding=1),
            nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            
            torch.nn.MaxPool2d(2),
            
            nn.Conv2d(64,128,2,padding=1),
            nn.BatchNorm2d(128),
            torch.nn.ReLU(),
            
            torch.nn.MaxPool2d(2),
            
            nn.Conv2d(128,256,2,padding=1),
            nn.BatchNorm2d(256),
            torch.nn.ReLU(),
            
            torch.nn.MaxPool2d(2),
            
            nn.Conv2d(256,512,2,padding=1),
            nn.BatchNorm2d(512),
            torch.nn.ReLU(),
            
            torch.nn.MaxPool2d(2),
            
            nn.Conv2d(512,1024,2,padding=1),
            nn.BatchNorm2d(1024),
            torch.nn.ReLU(),
            
            torch.nn.MaxPool2d(2),
            
            nn.Conv2d(1024,2048,2,padding=1),
            nn.BatchNorm2d(2048),
            torch.nn.ReLU(),
            
            torch.nn.MaxPool2d(2),
            
            nn.Conv2d(2048,4096,2,padding=1),
            nn.BatchNorm2d(4096),
            torch.nn.ReLU(),
            
            torch.nn.MaxPool2d(2),
            
            nn.Conv2d(4096,(4096*2),2,padding=1),
            nn.BatchNorm2d((4096*2)),
            torch.nn.ReLU(),
            
            torch.nn.MaxPool2d(2)
            
        )
        
        self.flatten = nn.Flatten()
        self.linearstack = nn.Sequential(
            nn.Linear(4096*2,100),
            nn.Sigmoid()
            )
        
    def forward(self,x):
        x = self.conv2d(x)
        flat = self.flatten(x)
        logits = self.linearstack(flat)
        return logits
model = NN().to(device)
print(model)

100%|██████████| 169001437/169001437 [00:16<00:00, 10162624.19it/s]


Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified
torch.Size([3, 32, 32])
torch.Size([10])
Using cuda device
NN(
  (conv2d): Sequential(
    (0): Conv2d(3, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kerne

In [2]:
for batch, (X, y) in enumerate(datagen):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        break
pred.shape

torch.Size([10, 100])

In [3]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [4]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [5]:
epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(datagen, model, loss_fn, optimizer)
    test(datatestgen, model, loss_fn)
print("Done!")

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Epoch 1
-------------------------------
loss: 4.621253  [   10/50000]
loss: 4.651838  [ 1010/50000]
loss: 4.589988  [ 2010/50000]
loss: 4.529504  [ 3010/50000]
loss: 4.427194  [ 4010/50000]
loss: 4.646763  [ 5010/50000]
loss: 4.487611  [ 6010/50000]
loss: 4.311981  [ 7010/50000]
loss: 4.386660  [ 8010/50000]
loss: 4.487893  [ 9010/50000]
loss: 4.304029  [10010/50000]
loss: 4.410135  [11010/50000]
loss: 4.298252  [12010/50000]
loss: 4.429020  [13010/50000]
loss: 4.358908  [14010/50000]
loss: 4.288771  [15010/50000]
loss: 4.279444  [16010/50000]
loss: 4.253261  [17010/50000]
loss: 4.244896  [18010/50000]
loss: 4.251626  [19010/50000]
loss: 4.387408  [20010/50000]
loss: 4.238271  [21010/50000]
loss: 4.245253  [22010/50000]
loss: 4.369299  [23010/50000]
loss: 4.339543  [24010/50000]
loss: 4.256835  [25010/50000]
loss: 4.452882  [26010/50000]
loss: 4.279565  [27010/50000]
loss: 4.317083  [28010/50000]
loss: 4.315490  [29010/50000]
loss: 4.279051  [30010/50000]
loss: 4.249997  [31010/50000]
